## SHREE KRISHNA KANTH S
## 225229136
## II MSc DATA SCIENCE "A"
## PDL LAB 12

### Step 1: Import the necessary libraries

In [1]:
import datetime
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D

### Step 2: Initialize some parameters

In [2]:
now = datetime.datetime.now
batch_size = 128
num_classes = 5
epochs = 5
img_rows, img_cols = 28, 28
filters = 32
pool_size = 2
kernel_size = 3

### Step 3: Partition MNIST dataset

In [3]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

### Create datasets for digits below 5 and digits 5 and above

In [4]:
x_train_1t5 = x_train[y_train < 5]
y_train_1t5 = y_train[y_train < 5]
x_test_1t5 = x_test[y_test < 5]
y_test_1t5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

### Step 4: Define the "feature" layers

In [5]:
feature_layers = [
    Conv2D(filters, kernel_size, padding='valid', activation='relu', input_shape=(img_rows, img_cols, 1)),
    Conv2D(filters, kernel_size, activation='relu'),
    MaxPooling2D(pool_size=(pool_size, pool_size)),
    Dropout(0.25),
    Flatten()
]

### Step 5: Define the "classification" layers

In [6]:
classification_layers = [
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
]

### Step 6: Define a Sequential model with feature and classification layers

In [7]:
model = Sequential(feature_layers + classification_layers)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                        

### Step 7: Create a function train_model()

In [8]:
def train_model(model, train_data, test_data, num_classes):
    x_train, y_train = train_data
    x_test, y_test = test_data
    
    x_train = x_train.reshape(-1, img_rows, img_cols, 1)
    x_test = x_test.reshape(-1, img_rows, img_cols, 1)
    x_train = x_train.astype('float32') / 255
    x_test = x_test.astype('float32') / 255
    
    print('x_train shape:', x_train.shape)
    print('x_test shape:', x_test.shape)
    
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])
    
    start_time = now()
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    end_time = now()
    print("Training time:", end_time - start_time)
    
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test loss:', score[0])
    print('Test accuracy:', score[1])

### Step 8: Training on digits 5 to 9

In [9]:
train_model(model, (x_train_gte5, y_train_gte5), (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
x_test shape: (4861, 28, 28, 1)
Epoch 1/5
230/230 [==============================] - 22s 91ms/step - loss: 1.5947 - accuracy: 0.2420 - val_loss: 1.5814 - val_accuracy: 0.3452
Epoch 2/5
230/230 [==============================] - 21s 91ms/step - loss: 1.5765 - accuracy: 0.2972 - val_loss: 1.5605 - val_accuracy: 0.4898
Epoch 3/5
230/230 [==============================] - 21s 92ms/step - loss: 1.5569 - accuracy: 0.3650 - val_loss: 1.5378 - val_accuracy: 0.5857
Epoch 4/5
230/230 [==============================] - 21s 91ms/step - loss: 1.5341 - accuracy: 0.4273 - val_loss: 1.5119 - val_accuracy: 0.6530
Epoch 5/5
230/230 [==============================] - 21s 92ms/step - loss: 1.5083 - accuracy: 0.4809 - val_loss: 1.4819 - val_accuracy: 0.6949
Training time: 0:01:45.941827
Test loss: 1.4818509817123413
Test accuracy: 0.6949187517166138


### Step 9: Freeze Feature Layers


In [10]:
for layer in feature_layers:
    layer.trainable = False

### Step 10: Print Summary

In [11]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 32)        0         
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 128)               589952    
                                                        

### Step 11: Training for digits 0 to 4 based on digits 5 to 9

In [12]:
train_model(model, (x_train_1t5, y_train_1t5), (x_test_1t5, y_test_1t5), num_classes)

x_train shape: (30596, 28, 28, 1)
x_test shape: (5139, 28, 28, 1)
Epoch 1/5
240/240 [==============================] - 6s 23ms/step - loss: 1.5928 - accuracy: 0.2443 - val_loss: 1.5678 - val_accuracy: 0.3345
Epoch 2/5
240/240 [==============================] - 5s 23ms/step - loss: 1.5582 - accuracy: 0.3233 - val_loss: 1.5315 - val_accuracy: 0.4491
Epoch 3/5
240/240 [==============================] - 6s 23ms/step - loss: 1.5249 - accuracy: 0.3934 - val_loss: 1.4959 - val_accuracy: 0.5454
Epoch 4/5
240/240 [==============================] - 5s 23ms/step - loss: 1.4912 - accuracy: 0.4646 - val_loss: 1.4597 - val_accuracy: 0.6270
Epoch 5/5
240/240 [==============================] - 5s 23ms/step - loss: 1.4574 - accuracy: 0.5300 - val_loss: 1.4230 - val_accuracy: 0.6943
Training time: 0:00:27.855789
Test loss: 1.4229519367218018
Test accuracy: 0.694298505783081
